In [11]:
#from selenium import webdriver
#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')
#driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

from bs4 import BeautifulSoup
import requests

import warnings
warnings.filterwarnings("ignore")

In [12]:
def get_soup(url):
    soup = BeautifulSoup(requests.get(url).text, "html.parser")
    return soup

In [13]:
Bundesländer = ['Baden-Württemberg', 'Brandenburg', 'Bremen', 'Hessen', 'Mecklenburg-Vorpommern', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 
                'Schleswig-Holstein', 'Bayern', 'Berlin', 'Hamburg', 'Niedersachsen', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Thüringen']

soup = get_soup("https://mein-monteurzimmer.de/")

url_list = {t.a.text:t.a['href'] for t in soup.find_all('div', {'class':'item'}) if t.a.text in Bundesländer}

In [14]:
import pandas as pd
result_df = pd.DataFrame(columns = ['Контактное лицо', 'Примечание Тег', 'Индекс', 'Город', 'Адрес (конкретный)', 'Адрес (короткий)', 'Рабочий телефон', 'Мобильный', 'Вебсайт', 'Email', 'Источник', 'Земля'])

earth = list(url_list.keys())[0]
earth_soup = get_soup("https://mein-monteurzimmer.de/"+url_list[earth])

for i in range(3):
    try:
        result_list = earth_soup.find_all('div', {'class': 'result-list'})[0].find_all('section')
        item = result_list[i]
        address = item.find('div', {'class':"item-details"}).find('div', {'itemprop': 'address'})

        item_soup = get_soup(item.find('a', {'itemprop':"url"})['href'])

        telephone = item_soup.find('section', {'itemprop':"contactPoint"}).find_all('dd', {'itemprop':"telephone"})

        try:
            v_1 = item_soup.find('section', {'itemprop':"contactPoint"}).find('dd', {'itemprop':"name"}).text 
        except:
            v_1 = None

        result_df = result_df.append({
            'Контактное лицо': v_1,
            'Примечание Тег' : item_soup.find('h1', {'itemprop':"name"}),
            'Индекс': address.find('span', {'class':"plz"}).text, 
            'Город': address.find('span', {'class':"city"}).text, 
            'Адрес (конкретный)': address.find('meta')['content'], 
            'Адрес (короткий)': ' '.join([address.find('meta')['content'], address.find('span', {'class':"plz"}).text,  address.find('span', {'class':"city"}).text, ]), 
            'Рабочий телефон': telephone[0].text, 
            'Мобильный': None,
            'Вебсайт': item_soup.find('a', {'class': 'o link'}).text, 
            'Email':telephone[1].text if len(telephone) != 1 else None,
            'Источник': item.find('a', {'itemprop':"url"})['href'],
            'Земля': earth
            }, ignore_index=True)
        print(f"OK    | {i} --- {item.find('a', {'itemprop':'url'})['href']}")
    except:
        try:
            print(f"ERROR | {i} --- {item.find('a', {'itemprop':'url'})['href']}")
        except:
            print(f"ERROR | {i} --- URL ERROR")

OK    | 0 --- https://mein-monteurzimmer.de/123002/monteurzimmer/berlin-berlin-potsdam-schoenefeld-gruenheide-oranienburg
OK    | 1 --- https://mein-monteurzimmer.de/122401/monteurzimmer/berlin-monteurkoenig-in-berlin-pro-firma-eine-wohnung-kein-firmen-mix
OK    | 2 --- https://mein-monteurzimmer.de/121690/monteurzimmer/berlin-juwel


In [15]:
result_df

,Контактное лицо,Примечание Тег,Индекс,Город,Адрес (конкретный),Адрес (короткий),Рабочий телефон,Мобильный,Вебсайт,Email,Источник,Земля
0,Herr Rico WalterStraße / HausnummerLinienstraß...,"[Berlin, Potsdam, Schönefeld, Grünheide, Orani...",10119,Berlin,Linienstraße 78,Linienstraße 78 10119 Berlin,020183886000Handy 2anfragen@tengo-internationa...,None,https://tengo-international.com/anfragen/,anfragen@tengo-international.comWebseitehttps:...,https://mein-monteurzimmer.de/123002/monteurzi...,Berlin
1,Herr Shabo DavidsStraße / HausnummerVoltairest...,[MonteurKönig in Berlin - Pro Firma eine Wohnu...,10179,Berlin,Voltairestraße 1,Voltairestraße 1 10179 Berlin,+491738088826Unterkunft Empfehlen Facebook Twi...,None,Berlin (1km),None,https://mein-monteurzimmer.de/122401/monteurzi...,Berlin
2,None,[JUWEL],10119,Berlin,Torstrasse 131,Torstrasse 131 10119 Berlin,0303244678Handy01725113620Unterkunft Empfehlen...,None,Berlin (1km),01725113620Unterkunft Empfehlen Facebook Twitt...,https://mein-monteurzimmer.de/121690/monteurzi...,Berlin


In [ ]:
for i in range(int(s.find('a', {'class': "o button secondary as-cube", 'rel':"noopener"}).text)):
    s = get_soup(f"https://mein-monteurzimmer.de/{url_list[earth]}?skip={i}")
    s = s.find_all('div', {'class': 'result-list'})[0].find_all('section')
    print(len(s))